In [ ]:
import laspy
import numpy as np
import time

In [ ]:
def goup_by_voxels(points, voxel_size: float):
    print("Grouping points into voxels...")
    # Присвоение каждой точке индекса в сетке
    mins = points.min(axis=0)
    idx = np.floor((points - mins) / voxel_size).astype(np.int32)

    # Группировка по вокселям:
    #  - unique_voxels - уникальные ячейки
    #  - inverse - индекс уникальной ячейки для каждой точки
    #  - counts - количество точек в ячейке
    unique_voxels, inverse, counts = np.unique(
        idx,
        axis=0,
        return_inverse=True,
        return_counts=True)
    
    return unique_voxels, inverse, counts

In [ ]:
def filter(unique_voxels, inverse, counts, n_points, min_points_in_voxel):
# Фильтр по минимальному кол-ву точек в ячейке
    valid_voxels = counts >= min_points_in_voxel
    valid_count = np.sum(valid_voxels)

    if valid_count == 0:
        raise ValueError("No voxels left after applying min_points_in_voxel filter")

    print(f"Voxels after min_points filter: {valid_count} (min = {min_points_in_voxel})")

    # Маска для точек
    mask_n = np.zeros(len(inverse), dtype=bool)

    mask_all = valid_voxels[inverse]

    valid_mask = valid_voxels[inverse]
    idx = np.where(valid_mask)[0]
    vox = inverse[valid_mask]

    order = np.lexsort((idx, vox))
    idx = idx[order]
    vox = vox[order]

    starts = np.r_[0, np.flatnonzero(vox[1:] != vox[:-1]) + 1]
    pos = np.arange(len(vox)) - np.repeat(starts, np.diff(np.r_[starts, len(vox)]))

    keep = pos < n_points
    keep_global = idx[keep]
    mask_n[keep_global] = True

    return mask_all, mask_n

In [ ]:
input_las_path = "data/test_data/non_ground.las"
output_path = "data/voxel_test"

print("Loading LAS...")
las = laspy.read(input_las_path)
points = np.vstack((las.x, las.y, las.z)).T.astype(np.float32)

N = las.header.point_count
print(f"{N} points loaded.")


voxel_sizes = []
point_numbers = []

for voxel_size in voxel_sizes:
    unique_voxels, inverse, counts = goup_by_voxels(points, voxel_size)

    for n in point_numbers:
        mask_all, mask_n = filter(unique_voxels, inverse, counts, n, n)

        print(f"filter:\n  voxel_size: {voxel_size}\n  number of points: {n}")
        print(f"  All remaining points: {mask_all.sum()}")
        print(f"  N remaining points: {mask_n.sum()}")
        print("   Creating new LAS...")
        new_las1 = las[mask_all]
        new_las2 = las[mask_n]
        print("   Writing file...")
        new_las1.write(f"{output_path}\{voxel_size}x{n}_all_points.las")
        new_las2.write(output_las_path)
        print("   Done.")